In [77]:
import sqlite3
import pandas as pd
from datetime import datetime, timedelta


In [79]:
# Kết nối đến database SQLite
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Tạo bảng student
cursor.execute('''
    CREATE TABLE student (
        student_id INTEGER PRIMARY KEY,
        name TEXT,
        class TEXT,
        course_id INTEGER,
        score REAL
    )
''')

# Tạo bảng course
cursor.execute('''
    CREATE TABLE course (
        id INTEGER PRIMARY KEY,
        course_name TEXT
    )
''')

# Thêm dữ liệu vào bảng student
students = [
    (1, 'Nguyen Minh Hoang', 'May Tinh', 12, 6.7),
    (2, 'Tran Thi Lan', 'Kinh Te', 34, 9.2),
    (3, 'Pham Van Nam', 'Toan Tin', None, 7.9),
    (4, 'Le Thanh Huyen', 'Toan Tin', 20, 7.2),
    (5, 'Vu Quoc Anh', 'May Tinh', 24, 8.0),
    (6, 'Dang Thuy Linh', 'May Tinh', 24, 5.5),
    (7, 'Bui Tien Dung', 'Kinh Te', 34, 9.2),
    (8, 'Ho Ngoc Mai', 'Toan Tin', 20, 8.8),
    (9, 'Duong Huu Phuc', 'Kinh Te', None, 7.2),
    (10, 'Cao Thi Hanh', 'May Tinh', None, 7.0)
]
cursor.executemany('INSERT INTO student VALUES (?, ?, ?, ?, ?)', students)

# Thêm dữ liệu vào bảng course
courses = [(12, 'Giai tich'), (34, 'Thong ke'), (26, 'Tin hoc')]
cursor.executemany('INSERT INTO course VALUES (?, ?)', courses)


In [80]:
# 1. Thực hiện JOIN
query = '''
    SELECT s.student_id, s.name, s.class, s.course_id, s.score, c.course_name
    FROM student s
    LEFT JOIN course c ON s.course_id = c.id
'''
result = pd.read_sql_query(query, conn)
print("LEFT JOIN Result:")
print(result)

# 1.1. Thực hiện tích Descartes
query_cartesian = "SELECT * FROM student, course;"
cartesian_result = pd.read_sql_query(query_cartesian, conn)
print("Cartesian Product Result:")
print(cartesian_result)


LEFT JOIN Result:
   student_id               name     class  course_id  score course_name
0           1  Nguyen Minh Hoang  May Tinh       12.0    6.7   Giai tich
1           2       Tran Thi Lan   Kinh Te       34.0    9.2    Thong ke
2           3       Pham Van Nam  Toan Tin        NaN    7.9        None
3           4     Le Thanh Huyen  Toan Tin       20.0    7.2        None
4           5        Vu Quoc Anh  May Tinh       24.0    8.0        None
5           6     Dang Thuy Linh  May Tinh       24.0    5.5        None
6           7      Bui Tien Dung   Kinh Te       34.0    9.2    Thong ke
7           8        Ho Ngoc Mai  Toan Tin       20.0    8.8        None
8           9     Duong Huu Phuc   Kinh Te        NaN    7.2        None
9          10       Cao Thi Hanh  May Tinh        NaN    7.0        None
Cartesian Product Result:
    student_id               name     class  course_id  score  id course_name
0            1  Nguyen Minh Hoang  May Tinh       12.0    6.7  12   Giai ti

In [81]:
# 2. Cập nhật course_id còn thiếu và loại bỏ dữ liệu không hợp lệ
cursor.execute("""
UPDATE student 
SET course_id = (SELECT MIN(id) FROM course) 
WHERE course_id IS NULL
""")
conn.commit()

cursor.execute("""
DELETE FROM student 
WHERE course_id NOT IN (SELECT id FROM course)
""")
conn.commit()

In [82]:
updated_students_df = pd.read_sql("SELECT * FROM student", conn)
print("\nDữ liệu sinh viên sau khi cập nhật course_id:")
print(updated_students_df)



Dữ liệu sinh viên sau khi cập nhật course_id:
   student_id               name     class  course_id  score
0           1  Nguyen Minh Hoang  May Tinh         12    6.7
1           2       Tran Thi Lan   Kinh Te         34    9.2
2           3       Pham Van Nam  Toan Tin         12    7.9
3           7      Bui Tien Dung   Kinh Te         34    9.2
4           9     Duong Huu Phuc   Kinh Te         12    7.2
5          10       Cao Thi Hanh  May Tinh         12    7.0


In [83]:
# 3. Tính toán tổng số sinh viên và điểm trung bình
summary_class = pd.read_sql_query('''
    SELECT class, COUNT(*) as total_students, AVG(score) as avg_score
    FROM student
    GROUP BY class
''', conn)
print("Summary by Class:")
print(summary_class)

summary_course = pd.read_sql_query('''
    SELECT course_id, COUNT(*) as total_students, AVG(score) as avg_score
    FROM student
    GROUP BY course_id
''', conn)
print("Summary by Course:")
print(summary_course)

Summary by Class:
      class  total_students  avg_score
0   Kinh Te               3   8.533333
1  May Tinh               2   6.850000
2  Toan Tin               1   7.900000
Summary by Course:
   course_id  total_students  avg_score
0         12               4        7.2
1         34               2        9.2


In [84]:
# 4. Phân loại sinh viên theo điểm
cursor.execute('ALTER TABLE student ADD COLUMN rank TEXT')
cursor.execute('''
    UPDATE student SET rank = CASE
        WHEN score >= 9.0 THEN 'Xuất sắc'
        WHEN score >= 6.0 THEN 'Tốt'
        ELSE 'Kém'
    END
''')
conn.commit()

In [85]:
# 5. Xếp hạng sinh viên theo điểm
query_top3 = 'SELECT * FROM student ORDER BY score DESC LIMIT 3'
query_bottom3 = 'SELECT * FROM student ORDER BY score ASC LIMIT 3'

top_students = pd.read_sql_query(query_top3, conn)
bottom_students = pd.read_sql_query(query_bottom3, conn)
print("Top 3 Students:")
print(top_students)
print("Bottom 3 Students:")
print(bottom_students)


Top 3 Students:
   student_id           name     class  course_id  score      rank
0           2   Tran Thi Lan   Kinh Te         34    9.2  Xuất sắc
1           7  Bui Tien Dung   Kinh Te         34    9.2  Xuất sắc
2           3   Pham Van Nam  Toan Tin         12    7.9       Tốt
Bottom 3 Students:
   student_id               name     class  course_id  score rank
0           1  Nguyen Minh Hoang  May Tinh         12    6.7  Tốt
1          10       Cao Thi Hanh  May Tinh         12    7.0  Tốt
2           9     Duong Huu Phuc   Kinh Te         12    7.2  Tốt


In [87]:
# 6. Kiểm tra và thêm trường graduation_date nếu chưa có
cursor.execute("PRAGMA table_info(student)")
columns = [row[1] for row in cursor.fetchall()]
if "graduation_date" not in columns:
    cursor.execute('ALTER TABLE student ADD COLUMN graduation_date TEXT')

current_date = datetime.now()

# Cập nhật ngày tốt nghiệp dựa trên xếp hạng điểm số
students = cursor.execute('SELECT student_id, score FROM student WHERE score IS NOT NULL ORDER BY score DESC').fetchall()

for rank, (student_id, score) in enumerate(students, start=1):
    grad_date = current_date + timedelta(days=rank * 30)
    cursor.execute('UPDATE student SET graduation_date = ? WHERE student_id = ?', (grad_date.strftime('%Y-%m-%d'), student_id))

conn.commit()

# Kiểm tra dữ liệu sau khi cập nhật
final_data = pd.read_sql_query('SELECT * FROM student', conn)
print("Final Student Data with Graduation Date:")
print(final_data)

Final Student Data with Graduation Date:
   student_id               name     class  course_id  score      rank  \
0           1  Nguyen Minh Hoang  May Tinh         12    6.7       Tốt   
1           2       Tran Thi Lan   Kinh Te         34    9.2  Xuất sắc   
2           3       Pham Van Nam  Toan Tin         12    7.9       Tốt   
3           7      Bui Tien Dung   Kinh Te         34    9.2  Xuất sắc   
4           9     Duong Huu Phuc   Kinh Te         12    7.2       Tốt   
5          10       Cao Thi Hanh  May Tinh         12    7.0       Tốt   

  graduation_date  
0      2025-09-09  
1      2025-04-12  
2      2025-06-11  
3      2025-05-12  
4      2025-07-11  
5      2025-08-10  


In [88]:
conn.close()
